In [155]:
import pandas as pd
import regex as re

dataset = pd.read_csv("train_trimmed.csv")
# display(dataset)

C:\Users\harry\AppData\Local\Temp\ipykernel_30400\2469928685.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("train_trimmed.csv")


In [156]:
#Removing data with missing values equal to Nan 
dataset = dataset.dropna()

#drop the ID column
dataset = dataset.drop('ID', axis=1)

dataset['Annual_Income'] = dataset['Annual_Income'].str.replace('_', '')
dataset['Num_of_Delayed_Payment'] = dataset['Num_of_Delayed_Payment'].str.replace('_', '')
dataset['Num_of_Loan'] = dataset['Num_of_Loan'].str.replace('_', '')
dataset["Outstanding_Debt"] = dataset["Outstanding_Debt"].str.replace('_', '')
dataset["Monthly_Balance"] = dataset["Monthly_Balance"].str.replace('_', '')





#Remove the underscores from the age data entries 
dataset['Age'] = dataset['Age'].str.replace('_', '')
# display(dataset.head())
dataset = dataset.dropna()


In [157]:
dataset['Age'] = dataset['Age'].astype(int)
dataset = dataset[(dataset['Age'] >= 0) & (dataset['Age'] < 100)]

#display the age range 
display(dataset['Age'].min())
display(dataset['Age'].max())

14

95

In [158]:
#Need to convert columns with numerical values and assert that they are all numerical
dataset.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

Age                          True
Annual_Income                True
Monthly_Inhand_Salary        True
Num_Bank_Accounts            True
Num_Credit_Card              True
Interest_Rate                True
Num_of_Loan                  True
Type_of_Loan                False
Num_of_Delayed_Payment       True
Num_Credit_Inquiries         True
Outstanding_Debt             True
Credit_Utilization_Ratio     True
Credit_History_Age          False
Total_EMI_per_month          True
Payment_Behaviour           False
Monthly_Balance              True
Credit_Score                False
dtype: bool

In [159]:
#Type converting all columns to numerical 
dataset['Annual_Income'] = dataset["Annual_Income"].astype(float)
dataset["Monthly_Balance"] = dataset["Monthly_Balance"].astype(float)
dataset["Monthly_Inhand_Salary"] = dataset["Monthly_Inhand_Salary"].astype(float)
dataset["Num_Bank_Accounts"] = dataset["Num_Bank_Accounts"].astype(int)
dataset["Num_Credit_Card"] = dataset["Num_Credit_Card"].astype(int)
dataset["Interest_Rate"] = dataset["Interest_Rate"].astype(float)
dataset["Num_of_Loan"] = dataset["Num_of_Loan"].astype(int)
dataset["Num_of_Delayed_Payment"] = dataset["Num_of_Delayed_Payment"].astype(int)
dataset["Outstanding_Debt"] = dataset["Outstanding_Debt"].astype(float)

#Manual Outlier removal 
dataset = dataset[dataset['Num_Bank_Accounts'] <= 13]
dataset = dataset[dataset['Num_Credit_Card'] <= 12]
dataset = dataset[dataset['Interest_Rate'] <= 19]
dataset = dataset[dataset['Num_of_Loan'] <= 8]
dataset = dataset[dataset['Num_of_Delayed_Payment'] <= 23]
dataset = dataset[dataset['Num_Credit_Inquiries'] <= 15]
dataset = dataset[dataset['Num_of_Delayed_Payment'] <= 18]

#Iterating and converting the credit history to numerical values  
for i in dataset['Credit_History_Age']:
    years = (re.findall(r'(\d+) Years', i))    
    months = (re.findall(r'(\d+) Months', i))
    total_months = int(years[0]) * 12 + int(months[0])
    dataset['Credit_History_Age'] = dataset['Credit_History_Age'].replace(i, total_months)

df_filtered = dataset 

df_filtered

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,19114.12,1824.843333,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,4.0,809.98,26.822620,265,49.574949,High_spent_Small_value_payments,312.494089,Good
6,23,19114.12,1824.843333,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4.0,809.98,22.537593,271,49.574949,Low_spent_Small_value_payments,244.565317,Good
8,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,4,2.0,605.03,24.464031,319,18.816215,Low_spent_Small_value_payments,470.690627,Standard
9,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,1,2.0,605.03,38.550848,320,18.816215,High_spent_Large_value_payments,484.591214,Good
12,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,1,2.0,605.03,34.977895,323,18.816215,Low_spent_Small_value_payments,444.867032,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98278,24,59868.93,5111.077500,4,7,9.0,4,"Not Specified, Mortgage Loan, Home Equity Loan...",9,4.0,417.72,39.666381,347,104.622038,!@9#%8,310.520171,Standard
98279,24,59868.93,5111.077500,4,7,9.0,4,"Not Specified, Mortgage Loan, Home Equity Loan...",11,4.0,417.72,24.803887,348,104.622038,Low_spent_Small_value_payments,80.190256,Standard
98298,38,41015.55,3152.962500,0,4,4.0,1,Payday Loan,1,2.0,599.40,30.760830,301,25.671257,Low_spent_Small_value_payments,178.327142,Standard
98299,38,41015.55,3152.962500,0,4,4.0,1,Payday Loan,1,2.0,599.40,31.936092,302,25.671257,Low_spent_Small_value_payments,192.083554,Standard


,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,19114.12,1824.843333,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,4.0,809.98,26.822620,265,49.574949,High_spent_Small_value_payments,312.494089,Good
6,23,19114.12,1824.843333,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4.0,809.98,22.537593,271,49.574949,Low_spent_Small_value_payments,244.565317,Good
8,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,4,2.0,605.03,24.464031,319,18.816215,Low_spent_Small_value_payments,470.690627,Standard
9,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,1,2.0,605.03,38.550848,320,18.816215,High_spent_Large_value_payments,484.591214,Good
12,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,1,2.0,605.03,34.977895,323,18.816215,Low_spent_Small_value_payments,444.867032,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98278,24,59868.93,5111.077500,4,7,9.0,4,"Not Specified, Mortgage Loan, Home Equity Loan...",9,4.0,417.72,39.666381,347,104.622038,!@9#%8,310.520171,Standard
98279,24,59868.93,5111.077500,4,7,9.0,4,"Not Specified, Mortgage Loan, Home Equity Loan...",11,4.0,417.72,24.803887,348,104.622038,Low_spent_Small_value_payments,80.190256,Standard
98298,38,41015.55,3152.962500,0,4,4.0,1,Payday Loan,1,2.0,599.40,30.760830,301,25.671257,Low_spent_Small_value_payments,178.327142,Standard
98299,38,41015.55,3152.962500,0,4,4.0,1,Payday Loan,1,2.0,599.40,31.936092,302,25.671257,Low_spent_Small_value_payments,192.083554,Standard


In [160]:
# Remove Rows with invalid data
columns = ["Annual_Income", "Num_of_Delayed_Payment", "Num_of_Loan", "Outstanding_Debt", "Monthly_Balance", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate", "Num_Credit_Inquiries", "Outstanding_Debt", "Total_EMI_per_month", "Credit_History_Age", "Credit_Utilization_Ratio"]

for col in columns:
    print(col)
    df_filtered = df_filtered.drop(df_filtered[df_filtered[col] < 0].index)

Annual_Income
Num_of_Delayed_Payment
Num_of_Loan
Outstanding_Debt
Monthly_Balance
Monthly_Inhand_Salary
Num_Bank_Accounts
Num_Credit_Card
Interest_Rate
Num_Credit_Inquiries
Outstanding_Debt
Total_EMI_per_month
Credit_History_Age
Credit_Utilization_Ratio


In [161]:
# One hot encoding for Type of loan 
# Find all unique regex matches with 'loan'
loan_matches = list(set(re.findall(r'\w+ Loan', ' '.join(dataset['Type_of_Loan']))))
print(loan_matches)

# Create a new column for each loan type
dataset['Auto Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Auto Loan', x) else 0)
dataset['Credit-Builder Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Credit-Builder Loan', x) else 0)
dataset['Personal Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Personal Loan', x) else 0)
dataset['Payday Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Payday Loan', x) else 0)
dataset['Student Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Student Loan', x) else 0)
dataset['Morgage Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Morgage Loan', x) else 0)
dataset['Home Equity Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Home Equity Loan', x) else 0)
dataset['Consolidation Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Consolidation Loan', x) else 0)

# Drop the original column
df_filtered = df_filtered.drop('Type_of_Loan', axis=1)


['Equity Loan', 'Student Loan', 'Builder Loan', 'Auto Loan', 'Mortgage Loan', 'Personal Loan', 'Payday Loan', 'Consolidation Loan']


In [162]:
# Look at dataframe
display(df_filtered.describe())

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Num_of_Delayed_Payment,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Monthly_Balance
count,30194.000000,3.019400e+04,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000,30194.000000
mean,34.522355,1.815995e+05,4743.053594,4.424919,4.836623,9.821554,3.032092,10.360568,4.262238,921.074054,32.501918,256.427734,1450.955063,428.990219
std,11.018166,1.432740e+06,3422.921539,2.366694,1.729602,5.097709,1.602584,4.867909,3.078530,697.997241,5.100806,86.750655,8394.464408,218.652051
min,14.000000,7.023160e+03,333.596667,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.230000,21.027665,3.000000,4.462837,0.131136
25%,25.000000,2.270994e+04,1926.528333,3.000000,4.000000,6.000000,2.000000,7.000000,2.000000,412.770000,28.343479,200.000000,38.053440,285.801285
50%,34.000000,4.173476e+04,3442.304167,4.000000,5.000000,9.000000,3.000000,11.000000,4.000000,832.030000,32.501463,255.000000,75.224655,361.846709
75%,43.000000,8.517270e+04,6978.081667,6.000000,6.000000,14.000000,4.000000,14.000000,6.000000,1266.310000,36.698722,329.000000,163.535779,521.238058
max,56.000000,2.410537e+07,15204.633333,11.000000,11.000000,19.000000,8.000000,18.000000,15.000000,4965.780000,49.564519,404.000000,82236.000000,1552.946094


In [163]:
# Perform one hot encoding for Payment_Behavior
df_filtered = df_filtered[df_filtered['Payment_Behaviour'] != '!@9#%8']

payment_encoded = pd.get_dummies(df_filtered['Payment_Behaviour'])
payment_encoded = payment_encoded.astype(int)
df_filtered = pd.concat([df_filtered, payment_encoded], axis=1)

print(df_filtered.columns.tolist())

df_filtered.to_csv("train_processed.csv", sep=',', index=False, encoding='utf-8')




['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score', 'High_spent_Large_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Small_value_payments', 'Low_spent_Large_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Small_value_payments']
